In [5]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')


Authenticated
Mounted at /content/drive


In [6]:
!pip install gspread --upgrade # Uninstall gspread-3.0.1 with 3.7.0
import gspread
from gspread.models import Cell, Spreadsheet
from oauth2client.client import GoogleCredentials as GC
import random
import string
from datetime import date, time, datetime, timedelta
gs_ = gspread.authorize(GC.get_application_default())

  Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [60]:
def getGworkbook(ref, client):
  """
  ref: is a gsheet url or key
  """
  try:
    return client.open_by_url(ref)
  except:
    return client.open_by_key(ref)

def genFileName(N):
  # import random
  # import string
  return ''.join(random.choice(string.ascii_uppercase + string.digits) for _ in range(N))

def genWeekEndingsList(initDate, n=52, ascending=True):
  ll = list()
  ll.append(initDate)
  if ascending:
    inc_=7
  else:
    inc_=-7
  for i in range(n):
    dateDiff = timedelta(days=inc_*(i+1))
    date2Append = initDate+dateDiff
    ll.append(date2Append)
  return ll

# def addSheetsToSpreadsheet(gs_spreadsheet,newSheetNames,source_sheet_id=0):
#   for i in range(1,len(newSheetNames)):
#     gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=newSheetNames[i-1])


def addSheetToSpreadsheet(gs_spreadsheet,source_sheet_id, insert_sheet_index, new_sheet_id, new_sheet_name):
  gs_spreadsheet.duplicate_sheet(source_sheet_id=source_sheet_id, insert_sheet_index=insert_sheet_index, new_sheet_id=new_sheet_id, new_sheet_name=new_sheet_name)

def getStoreListGsheet(gSheetID, gspreadAuth):
  # get store list gsheet
  import numpy as np
  storeList_worksheet = getGworkbook(gSheetID, gspreadAuth).sheet1
  storeList_array = np.array(storeList_worksheet.get_all_values())[1:]

  store_nums = storeList_array.T[0]
  store_gsheet_names = storeList_array.T[1]
  store_name_in_gsheet = storeList_array.T[2]

  # clean access names:
  access_name_separator = ';'
  cleaned_perm_emails = [x[3].split(access_name_separator) for x in storeList_array]

  ll = []
  for i in range(len(cleaned_perm_emails)):
    ll.append( [s.strip() for s in cleaned_perm_emails[i]] )
  cleaned_perm_emails = ll

  return store_nums, store_gsheet_names, cleaned_perm_emails

def protectSheetExceptSpecificRange(spreadsheet, sheetId, CanEditEmailAddress):
  # https://stackoverflow.com/questions/67896599/gspread-protect-sheet-except-certain-cells
  body = {
      "requests": [
          {
              "addProtectedRange": {
                  "protectedRange": {
                      "range": {
                          "sheetId": sheetId
                      },
                      "unprotectedRanges": [
                          { # c6:c26
                              "sheetId": sheetId,
                              "startRowIndex": 5,
                              "endRowIndex": 26,
                              "startColumnIndex": 2,
                              "endColumnIndex": 3
                          },
                          { # f4
                              "sheetId": sheetId,
                              "startRowIndex": 3,
                              "endRowIndex": 4,
                              "startColumnIndex": 5,
                              "endColumnIndex": 6
                          },
                          { # f6:f12
                              "sheetId": sheetId,
                              "startRowIndex": 5,
                              "endRowIndex": 12,
                              "startColumnIndex": 5,
                              "endColumnIndex": 6
                          },
                          { # f21:f29
                              "sheetId": sheetId,
                              "startRowIndex": 20,
                              "endRowIndex": 29,
                              "startColumnIndex": 5,
                              "endColumnIndex": 6
                          }
                      ],
                      "editors": {
                          "domainUsersCanEdit": False,
                          "users": CanEditEmailAddress #[emailAddress]
                      },
                      "warningOnly": False
                  }
              }
          }
      ]
  }
  spreadsheet.batch_update(body)



In [61]:
## inputs
templateGsheetID = '1LDSuhdmfHFidG_V46nCfx2GqFax6UvUPLGR8DtB7nNc' # this is the ID of the template gsheet
initial_date = date(year=2022, month=6, day=28)

storeListGsheetID = '1U4O_jC2UGYS636Oldr5KFmjML9rTNJBJF6TYybmqBbw' # this is the store list gsheet ID - with store numbers, store names and permission emails

# range2protect = ['A1:A29', 
#                  'B1:B29',
#                  'C1:C5', 'C27:C29',
#                  'D1:D29',
#                  'E1:E29',
#                  'F1:F3', 'F5', 'F13:F20']

we_dates = genWeekEndingsList(initial_date,n=52, ascending=False)
we_dates_str = [str(x) for x in we_dates]

peepsWithEditAccess = ['kherrmann@woolworths.com.au', 'kmaidment@woolworths.com.au','lagius@woolworths.com.au','mmarino@woolworths.com.au']

In [62]:
# get template gsheet
template_wb = getGworkbook(templateGsheetID, gs_)

## get store list gsheet
store_nums, store_gsheet_names, cleaned_perm_emails = getStoreListGsheet(gSheetID=storeListGsheetID, gspreadAuth=gs_)


In [63]:
# Create a new template file:
# add a new sheet 1 for each week
# and update + protect range

# create new template sheet which will be correctly structured and will act as our new template to duplicate later (instead of running entire process for each stores gsheet)
client = gspread.authorize(GC.get_application_default())

fName = "script_generated_template_file"
## create new file as copy of template
newTemplateGsheet = client.copy(templateGsheetID, title=fName, copy_permissions=True)

for i in range(1,len(we_dates_str)):
  i_we_dates_str = we_dates_str[i-1]
  addSheetToSpreadsheet(gs_spreadsheet=newTemplateGsheet,source_sheet_id=0, insert_sheet_index=i, new_sheet_id=i, new_sheet_name=i_we_dates_str)
  worksheet = newTemplateGsheet.worksheet(i_we_dates_str)

  # update date in sheet
  worksheet.update_acell('F2', i_we_dates_str)

  # # protect range:
  # for pc in range(len(range2protect)):
  #   i_range2protect = range2protect[pc]
  #   worksheet.add_protected_range(i_range2protect)

  protectSheetExceptSpecificRange(spreadsheet=newTemplateGsheet, sheetId=worksheet.id, CanEditEmailAddress=peepsWithEditAccess)
  
# delete Sheet1 from the new template gsheet
newTemplateGsheet.del_worksheet(newTemplateGsheet.sheet1)



{'replies': [{}],
 'spreadsheetId': '1NqIqVIjiVcSQTpzKvP3ZdFfrIp2DOYIWvKM4wdWbZkY'}

In [64]:
# Duplicate new template workbook per store
# plus share it

for sn in range(len(store_nums)):
  c_store_nums, c_store_gsheet_names, c_cleaned_perm_emails = store_nums[sn], store_gsheet_names[sn], cleaned_perm_emails[sn]
  
  fName = c_store_gsheet_names
  ## create new file as copy of template
  #newGsheet = client.copy(newGsheet, title=fName, copy_permissions=True)
  tempSiteSheet = client.copy(newTemplateGsheet.id, title=fName, copy_permissions=True)

  for i in range(1,len(we_dates_str)):
    i_we_dates_str = we_dates_str[i-1]
    worksheet = tempSiteSheet.worksheet(i_we_dates_str)
    # update store number in sheet
    worksheet.update_acell('B3', c_store_nums)
    #protectSheetExceptSpecificRange(spreadsheet=tempSiteSheet, sheetId=worksheet.id, CanEditEmailAddress=c_cleaned_perm_emails)
  
  # share the sheet
  for permUser in range(len(c_cleaned_perm_emails)):
    currentUser = c_cleaned_perm_emails[permUser]
    worksheet.spreadsheet.share(currentUser, perm_type='user', role='writer', notify=True, email_message=None, with_link=False)



KeyboardInterrupt: ignored